In [6]:
import os
from PIL import Image
import torch
from torchvision import models, transforms
import torch.nn as nn
from annoy import AnnoyIndex

images_folder = "C:/Users/atitk/PycharmProjects/230241_AtitKumarSatsangi_deeploycv/.venv/Cat"
images = os.listdir(images_folder)
weights=models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)
model.fc=nn.Identity()
print(model)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

annoy_index=AnnoyIndex(512,'angular')
for i in range(len(images)):
    image = Image.open(os.path.join(images_folder, images[i]))
    input_tensor = transform(image).unsqueeze(0)

    if input_tensor.size()[1] == 3:  # Check if the image has 3 channels (RGB)
        output_tensor = model(input_tensor)
        # predicted_class = torch.argmax(output_tensor)
        # print(f'{images[i]} predicted as {weights.meta["categories"][predicted_class]}')
        annoy_index.add_item(i,output_tensor[0])

        if i%100==0:
            print(f'Processing image {i}')
annoy_index.build(10)
annoy_index.save('cat_index.ann')




ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

True

In [13]:
import os
from PIL import Image,ImageDraw
import torch
from torchvision import models, transforms
import torch.nn as nn
from annoy import AnnoyIndex
weights=models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)
model.fc=nn.Identity()
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
annoy_index=AnnoyIndex(512,'angular')
annoy_index.load('cat_index.ann')
image_grid=Image.new('RGB',(1000,1000))
for i in range(len(images)):
    image = Image.open(os.path.join(images_folder, images[i]))
    input_tensor = transform(image).unsqueeze(0)

    if input_tensor.size()[1] == 3:  # Check if the image has 3 channels (RGB)
        output_tensor = model(input_tensor)
        nns=annoy_index.get_nns_by_vector(output_tensor[0],24)
        image=image.resize((200,200))
        image_draw=ImageDraw.Draw(image)
        image_draw.rectangle([(0,0),(199,199)],outline='red',width=8)
        image_grid.paste(image,((0,0)))

        for j in range(24):
            search_image=Image.open(os.path.join(images_folder, images[nns[j]]))
            search_image=search_image.resize((200,200))
            image_grid.paste(search_image,((200*((j+1)%5),200*((j+1)//5))))
        image_grid.save(f"ImageDump/image{i}.png")


KeyboardInterrupt: 